<a href="https://colab.research.google.com/github/epadam/Machine-Learning-Tutorial-Demo-Resources/blob/master/notebooks/nlp/document%20classification/news_classification_Bert_tf_idf_glove_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [ ]:
import torch
from datasets import load_dataset

In [ ]:
train_data = load_dataset('ag_news', split="train[:20000]")
test_data = load_dataset('ag_news', split="test")

Downloading:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/751k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_data = train_data.map(lambda x: {'labels': x['label']}, batched=True)
test_data = test_data.map(lambda x: {'labels': x['label']}, batched=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
model_name ='bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=train_data.features['label'].num_classes)

In [ ]:
train_data = train_data.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)
test_data = test_data.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=512), batched=True)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average ='micro')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy':acc,
      'f1':f1,
      'precision':precision,
      'recall': recall
  }

In [ ]:
training_args = TrainingArguments(
    output_dir = './',
    learning_rate = 3e-4,
    num_train_epochs = 1,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    logging_dir = './logs',
    logging_steps=100,
    do_train = True,
    do_eval = True,
    no_cuda= False,
    load_best_model_at_end= True,
    eval_steps=100,
    evaluation_strategy='steps'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

train_out=trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 120000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000


Step,Training Loss,Validation Loss


In [ ]:
model = model.cpu()

In [ ]:
from transformers import pipeline